In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import altair as alt

In [2]:
interaction_types = ['integrated_hessians',
                     'expected_hessians',
                     'hessians',
                     'hessians_times_inputs',
                     'shapley_sampling',
                     'contextual_decomposition',
                     'neural_interaction_detection']

In [3]:
dataset = 'simulated_10'

In [4]:
base_dir = 'results/' + dataset + '_{}.csv'

In [5]:
df_list = []
for interaction_type in interaction_types:
    df = pd.read_csv(base_dir.format(interaction_type))
    perf_all_interactions = df['mean_perf'].values[0]
    perf_no_interactions  = df['mean_perf'].values[-1]
    perf_denom = perf_no_interactions - perf_all_interactions

    df['mean_perf'] = (df['mean_perf'] - perf_all_interactions) / perf_denom
    df['sd_perf'] = (df['sd_perf'] - perf_all_interactions) / perf_denom
    df_list.append(df)

In [6]:
df_total = pd.concat(df_list)

In [ ]:
line = alt.Chart(df_total, width=1000, height=600).mark_line().transform_filter(
    'datum.num_interactions_removed < 10'
).encode(
    x=alt.X('num_interactions_removed:O', title='Number of Interactions Removed'),
    y=alt.Y('mean_perf:Q', title='Mean Error Relative to Baseline'),
    color=alt.Color('interaction_type:N')
)
band = alt.Chart(df_total).transform_calculate(
    ymin="datum.mean_perf - datum.sd_perf",
    ymax="datum.mean_perf + datum.sd_perf"
).mark_errorbar().encode(
    x=alt.X('num_interactions_removed:O'),
    y="ymin:Q",
    y2="ymax:Q",
    color=alt.Color('interaction_type:N')
)
line

alt.Chart(...)